In [26]:
# CREATE NODES

import pandas as pd

# CSV-Datei laden
csv_path = r'C:\Users\Julius Guizetti\IdeaProjects\matsim-hamburg-HA2\scenarios\input\stationen_ba.csv'  # Pfad zur CSV-Datei anpassen
df = pd.read_csv(csv_path, sep=';')

# Neue XML-Datei erstellen
output_path = r'C:\Users\Julius Guizetti\IdeaProjects\matsim-hamburg-HA2\scenarios\input\adapt_network_TEST.xml'  # Pfad zur Ausgabedatei anpassen

# XML-Datei schreiben
with open(output_path, 'w', encoding='utf-8') as f:
    f.write('<network>\n')
    f.write('    <nodes>\n')
    for _, row in df.iterrows():
        f.write(f'        <node id="{row["NodeID"]}" x="{row["x"]}" y="{row["y"]}">\n')
        f.write('        </node>\n')
    f.write('    </nodes>\n')
    f.write('</network>\n')

print(f"Nodes XML wurde erfolgreich unter {output_path} gespeichert.")


Nodes XML wurde erfolgreich unter C:\Users\Julius Guizetti\IdeaProjects\matsim-hamburg-HA2\scenarios\input\adapt_network.xml gespeichert.


In [3]:
import pandas as pd
import xml.etree.ElementTree as ET

# Bestehendes XML-Dokument laden
input_path = r'C:\Users\Julius Guizetti\IdeaProjects\matsim-hamburg-HA2.1\scenarios\input\adapt_network.xml'  # Pfad zur bestehenden XML-Datei anpassen
tree = ET.parse(input_path)
root = tree.getroot()

# CSV-Dateien laden
csv_path_ba = r'C:\Users\Julius Guizetti\IdeaProjects\matsim-hamburg-HA2.1\scenarios\input\stationen_ba.csv'  # Pfad zur CSV-Datei anpassen
df_ba = pd.read_csv(csv_path_ba, sep=';')

csv_path_ab = r'C:\Users\Julius Guizetti\IdeaProjects\matsim-hamburg-HA2.1\scenarios\input\stationen_ab.csv'  # Pfad zur CSV-Datei anpassen
df_ab = pd.read_csv(csv_path_ab, sep=';')

# Links-Element hinzufügen, falls nicht vorhanden
links_element = root.find('links')
if links_element is None:
    links_element = ET.SubElement(root, 'links')

# Funktion zum Erstellen von Links
def create_links(df, direction):
    for i in range(len(df)):
        # Link von einem Node zu sich selbst
        row = df.iloc[i]
        ET.SubElement(links_element, 'link', 
                      id=row['LinkID'], 
                      from_=row['NodeID'], 
                      to=row['NodeID'],
                      length="10.0",
                      freespeed="100",
                      capacity="10000",
                      permlanes="1.0",
                      oneway="1",
                      modes="pt")
        
        # Link zum darauffolgenden Node, falls vorhanden
        if i < len(df) - 1:
            from_node = df.iloc[i]['NodeID']
            to_node = df.iloc[i + 1]['NodeID']
            link_id = f"{direction}-{from_node}_to_{to_node}"
            ET.SubElement(links_element, 'link', 
                          id=link_id, 
                          from_=from_node,
                          to=to_node,
                          length = str(df.iloc[i]['distanceToNextNode']),
                          freespeed="100",
                          capacity="10000",
                          permlanes="1.0",
                          oneway="1",
                          modes="pt")

# Links für beide CSV-Dateien erstellen
create_links(df_ba, "BA")
create_links(df_ab, "AB")

# Funktion zum Formatieren des XML-Dokuments
def format_xml(element, level=0):
    indent = "    "
    if len(element):
        if not element.text or not element.text.strip():
            element.text = "\n" + indent * (level + 1)
        if not element.tail or not element.tail.strip():
            element.tail = "\n" + indent * level
        for elem in element:
            format_xml(elem, level + 1)
        if not element.tail or not element.tail.strip():
            element.tail = "\n" + indent * level
    else:
        if level and (not element.tail or not element.tail.strip()):
            element.tail = "\n" + indent * level

# XML-Dokument formatieren
format_xml(root)

# Geänderte XML-Datei speichern
output_path = r'C:\Users\Julius Guizetti\IdeaProjects\matsim-hamburg-HA2.1\scenarios\input\adapt_network_V2.xml'  # Pfad zur Ausgabedatei anpassen
tree.write(output_path, encoding='utf-8', xml_declaration=True)

print(f"Updated network XML wurde erfolgreich unter {output_path} gespeichert.")


Updated network XML wurde erfolgreich unter C:\Users\Julius Guizetti\IdeaProjects\matsim-hamburg-HA2.1\scenarios\input\adapt_network_V2.xml gespeichert.
